In [7]:
import numpy as np 
import torch
import random
import os

# customized 
from psml_defense import PsmlDefenseProxy

In [8]:
pareto_front_models = [
    'vgg11_bn',
    'vgg13_bn',
    'vgg16_bn',
    'vgg19_bn',
    'vit_small_patch16_384',
    'vit_base_patch16_384',
    'vit_large_patch16_384'
]

pareto_front_spec = [
	[70.44, 10.97], # vgg11_bn
	[74.02, 12.94], # vgg13_bn
	[74.68, 16.6 ], # vgg16_bn
	[75.16, 20.24], # vgg19_bn
	[90.04, 36.16], # vit_small_patch16_384
	[90.1 , 39.29], # vit_base_patch16_384
	[90.82, 73.69]  # vit_large_patch16_384
]

psml_pareto_front = np.array(pareto_front_spec)

In [9]:
eps = 100

In [10]:
defense = PsmlDefenseProxy(pareto_front_models=pareto_front_models, pareto_front_spec=pareto_front_spec, eps=eps, sensitivity=1)

Initializing PsmlDefenseProxy... w/ eps 100, sensitivity 1
vgg11_bn [70.44, 10.97]
vgg13_bn [74.02, 12.94]
vgg16_bn [74.68, 16.6]
vgg19_bn [75.16, 20.24]
vit_small_patch16_384 [90.04, 36.16]
vit_base_patch16_384 [90.1, 39.29]
vit_large_patch16_384 [90.82, 73.69]


In [11]:
defense.pareto_front

array([[70.44, 10.97],
       [74.02, 12.94],
       [74.68, 16.6 ],
       [75.16, 20.24],
       [90.04, 36.16],
       [90.1 , 39.29],
       [90.82, 73.69]])

In [12]:
defense.pareto_front_map

{(70.44, 10.97): 'vgg11_bn',
 (74.02, 12.94): 'vgg13_bn',
 (74.68, 16.6): 'vgg16_bn',
 (75.16, 20.24): 'vgg19_bn',
 (90.04, 36.16): 'vit_small_patch16_384',
 (90.1, 39.29): 'vit_base_patch16_384',
 (90.82, 73.69): 'vit_large_patch16_384'}

In [21]:
#all_epsilons = [1,2,3,4,5,10,50,100,500,1000]
all_epsilons = [0.01, 0.05, 0.1, 0.2, 0.5, 1,5,10, 20, 30, 50,100,]

In [22]:
# queries = [(random.randint(50, 100), random.randint(10, 250)) for _ in range(20)]
# queries = [(72.18, 18.5) for _ in range(20)] # vgg19
queries = [(98.02, 35.86) for _ in range(4000)] # vit_small_patch16_384

In [23]:
from collections import Counter

for eps in all_epsilons: 
    server = PsmlDefenseProxy(pareto_front_models=pareto_front_models, pareto_front_spec=pareto_front_spec, eps=eps, sensitivity=1)
    
    print(f"\ncomputing goodput for eps={eps}...")
    utility = 0
    answered_queries = 0
    selected_points = [] 
    selected_models = [] 
    
    for query in queries:
        # selected_element: (accuracy, latency)
        selected_element = server.l1_permute_and_flip_mechanism(query)
        # we can get none when the query is infeasible, then no point will have a high enough prob to be selected
        # print(query, ' -> ', selected_element)
        if selected_element is not None:
            answered_queries += 1
            selected_points.append(selected_element)
            selected_models.append(server.m_query(selected_element))
            
            utility += server.l1_score(float(selected_element[0]), float(selected_element[1]), query[0], query[1])
    
    print('utility score: ', utility / answered_queries)
    # for point, model in zip(selected_points, selected_models):
    #    print(point, ' -> ', model.__class__.__name__)
    
    model_counts = Counter([model for model in selected_models])
    total_models = sum(model_counts.values())  # Total selections
    
    # Calculate and display proportions
    print("model selection:")
    for model_name, count in model_counts.items():
        proportion = count / total_models
        print(f"{model_name}: {count} ({proportion:.2%})")

Initializing PsmlDefenseProxy... w/ eps 0.01, sensitivity 1
vgg11_bn [70.44, 10.97]
vgg13_bn [74.02, 12.94]
vgg16_bn [74.68, 16.6]
vgg19_bn [75.16, 20.24]
vit_small_patch16_384 [90.04, 36.16]
vit_base_patch16_384 [90.1, 39.29]
vit_large_patch16_384 [90.82, 73.69]

computing goodput for eps=0.01...
utility score:  0.8841768749999652
model selection:
vgg19_bn: 568 (14.20%)
vit_small_patch16_384: 568 (14.20%)
vit_large_patch16_384: 565 (14.12%)
vit_base_patch16_384: 577 (14.42%)
vgg13_bn: 634 (15.85%)
vgg16_bn: 529 (13.23%)
vgg11_bn: 559 (13.98%)
Initializing PsmlDefenseProxy... w/ eps 0.05, sensitivity 1
vgg11_bn [70.44, 10.97]
vgg13_bn [74.02, 12.94]
vgg16_bn [74.68, 16.6]
vgg19_bn [75.16, 20.24]
vit_small_patch16_384 [90.04, 36.16]
vit_base_patch16_384 [90.1, 39.29]
vit_large_patch16_384 [90.82, 73.69]

computing goodput for eps=0.05...
utility score:  0.8816842999999633
model selection:
vit_large_patch16_384: 623 (15.57%)
vgg16_bn: 548 (13.70%)
vgg11_bn: 584 (14.60%)
vgg13_bn: 548 (13